In [1]:
import pandas as pd
# import re
from datetime import datetime
import string

In [13]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df_raw):
    df = df_raw.copy()
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [3]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

FileNotFoundError: [Errno 2] No such file or directory: 'veic_2019_02.csv_amostra.txt'

# movimento_OCR_JAN_2019.txt_amostra.txt

In [37]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw.copy())
df_ocr_split.head()

,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900


# Tratando OCR

In [38]:
def get_datahora(string):
    date = string[38:52]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_placa(string):
    placa = string[0:7]
    return placa.strip()

df_ocr = pd.DataFrame()
df_ocr['CODCET'] = df_ocr_raw['x'].apply(lambda x: x[108:118])
df_ocr['DATAHORA'] = df_ocr_raw['x'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_raw['x'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_raw['x'].apply(lambda x: get_velocidade(x))
df_ocr['CGT'] = df_ocr_raw['x'].apply(lambda x: x[108:111])
df_ocr['LATITUDE'] = df_ocr_raw['x'].apply(lambda x: x[216:225])
df_ocr['LONGITUDE'] = df_ocr_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_ocr.to_csv('amostra_movimento_tratada.csv', sep=';', index=False)
df_ocr.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
87,0410291112,2019-01-01 00:00:40,KWA6203,46,041,-22.88229,-43.52213
34,0410041112,2019-01-01 00:00:12,LQA4260,32,041,-22.99365,-43.30290
45,0410552121,2019-01-01 00:00:18,LTG3261,28,041,-23.00726,-43.43819
63,0410191111,2019-01-01 00:00:27,LQW2315,39,041,-22.93380,-43.18294
44,0410491111,2019-01-01 00:00:18,KWQ3930,36,041,-22.97950,-43.58871


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [39]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca_raw = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_transcarioca_split = separar_colunas_df(df_transcarioca_raw.copy())
df_transcarioca_split.head(5)

,a,b,c,d,e,f
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


# Tratamento SCP_TRANSCARIOCA

In [40]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_velocidade(string):
    vel = string[92:94]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_latitude(string):
    lat = string[216:219]+"."+string[220:225]
    return lat

def get_longitude(string):
    lon = string[225:228]+"."+string[229:]
    return lon

df_transcarioca = pd.DataFrame()
df_transcarioca['CODCET'] = df_transcarioca_raw['x'].apply(lambda x:x[108:118])
df_transcarioca['DATAHORA'] = df_transcarioca_raw['x'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca_raw['x'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca_raw['x'].apply(lambda x: get_velocidade(x))
df_transcarioca['CGCT'] = df_transcarioca_raw['x'].apply(lambda x:x[108:111])
df_transcarioca['LATITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_longitude(x))

# Salvando em csv
df_transcarioca.to_csv('amostra_transcarioca_tratada.csv', sep=';', index=False)
df_transcarioca.head(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGCT,LATITUDE,LONGITUDE
0,0440021120,2019-03-01 00:10:31,,9,044,-22.87280,-43.33629
1,0440021120,2019-03-01 00:10:44,,24,044,-22.87280,-43.33629
2,0440021120,2019-03-01 00:10:44,,19,044,-22.87280,-43.33629
3,0440021120,2019-03-01 00:10:46,,28,044,-22.87280,-43.33629
4,0440021120,2019-03-01 00:10:48,,30,044,-22.87280,-43.33629
5,0440021120,2019-03-01 00:10:50,,30,044,-22.87280,-43.33629
6,0440021120,2019-03-01 00:10:58,,34,044,-22.87280,-43.33629
7,0440021120,2019-03-01 00:11:07,,32,044,-22.87280,-43.33629
8,0440021120,2019-03-01 00:11:08,,33,044,-22.87280,-43.33629
9,0440021120,2019-03-01 00:11:11,,34,044,-22.87280,-43.33629


# SPLICE_007201820190205.txt_amostra.txt

In [41]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_splice_raw = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)

# Para facilitar o tratamento
df_splice_raw.iloc[:,0] = df_splice_raw.iloc[:,0].replace({'( ){2,} ':'|'},regex=True) #df.iloc[(range linhas),(range colunas)]
df_splice_raw.iloc[:,0] = df_splice_raw.iloc[:,0].apply(lambda x: correctSplit(x))
df_splice_raw.head()

df_splice_raw.head()

,x
0,1MOTO|2019010100000105900RD3389|04503811120072...
1,1MOTO|2019010100000305500RD3381|00450541110072...
2,1MOTO|2019010100000505800RD3384|04504411120072...
3,1MOTO|2019010100002203900RD3414|04506811110072...
4,1MOTO|2019010100002205800RD3385|04504511120072...


In [42]:
df_splice = pd.DataFrame()

def get_datahora(string):
    date = string.split('|')[1][:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_Lat(string):
    lat_long = string.split('|')[-1]
    lat = '-' + lat_long.split('-')[1]
    return lat

def get_Long(string):
    lat_long = string.split('|')[-1]
    long = '-'+lat_long.split('-')[2]
    return long


df_splice['TIPO'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[0][:1])
df_splice['DATAHORA'] = df_splice_raw.iloc[:,0].apply(lambda x: get_datahora(x))
df_splice['CODCET'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[2][:10]) 
df_splice['VELOCIDADE'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[1][15:17])
df_splice['LATITUDE'] = df_splice_raw.iloc[:,0].apply(lambda x: get_Lat(x))
df_splice['LONGITUDE'] = df_splice_raw.iloc[:,0].apply(lambda x: get_Long(x))

# não foi encontrada nenhuma fatia que possuisse a placa do veículo, tampouco uma exclusiva do CGT
# perguntas: 1- é possível não haver placa?
#           2- o CGT é sempre igual aos primeiros 3 dígitos do CODCET?

In [43]:
df_splice.to_csv('amostra_splice_tratada.csv', sep=';', index=False)
df_splice.head(10)

,TIPO,DATAHORA,CODCET,VELOCIDADE,LATITUDE,LONGITUDE
0,1,9010-01-20 10:00:00,0450381112,59,-22.98468,-43.21814
1,1,9010-01-20 10:00:00,0045054111,55,-22.95759,-43.19957
2,1,9010-01-20 10:00:00,0450441112,58,-22.98537,-43.20337
3,1,9010-01-20 10:00:02,0450681111,39,-22.98475,-43.20046
4,1,9010-01-20 10:00:02,0450451112,58,-22.98481,-43.20567
5,1,9010-01-20 10:00:02,0450251112,43,-22.95417,-43.1917
6,1,9010-01-20 10:00:02,0450391112,42,-22.98459,-43.22075
7,1,9010-01-20 10:00:02,0450591112,31,-22.98431,-43.22281
8,1,9010-01-20 10:00:03,0450401112,69,-22.98545,-43.22315
9,1,9010-01-20 10:00:03,0450311112,32,-22.94885,-43.18549


# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [44]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech_raw = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
df_fiscaltech = separar_colunas_df(df_fiscaltech_raw)


# Extraindo partes onde possui informação requisitada
df_fiscaltech['b0'] = df_fiscaltech.b
df_fiscaltech['d'] = df_fiscaltech.c # aqui existe latidude e longitude, além de um número prévio que também poderia sar o 
#código do tipo
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[2][:17])

df_fiscaltech.head(4)

,a,b,c,b0,d
0,1,0106201903270901800FSCII2175,04200611130042016,0106201903270901800FSCII2175 0420061113004201...,3-23000409-43334161
1,1,0106201901164305100FSCII2214,04201712110042016,0106201901164305100FSCII2214 0420171211004201...,1
2,1,0106201901120105000FSCII2174,04200711240042016,0106201901120105000FSCII2174 0420071124004201...,4-23001615-43328645
3,1,0106201903304702100FSCII2175,04200611140042016,0106201903304702100FSCII2175 0420061114004201...,4-23000409-43334161


# Tratamento FISCALTECH

In [45]:
def get_datahora(string):
    date = string[:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_lat(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[1][:2]+'.'+string.split('-')[1][2:]

def get_long(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[2][:2]+'.'+string.split('-')[2][2:]

def get_placa(string):
    if len(string)==1:
        return ''
    else:
        return string[:7]

def get_tipo(string):
    if len(string)==1:
        return string
    else:
        return string[7:8]


#Separação de colunas 

df_fiscaltech['PLACA'] = df_fiscaltech.a.apply(lambda x: get_placa(x))
df_fiscaltech['TIPO'] = df_fiscaltech.a.apply(lambda x: get_tipo(x))
df_fiscaltech['CODCET'] = df_fiscaltech.c.apply(lambda x: x[:10])
df_fiscaltech['CGT'] = df_fiscaltech.c.apply(lambda x: x[:3])
df_fiscaltech['DATAHORA'] = df_fiscaltech.b.apply(lambda x: get_datahora(x))
df_fiscaltech['VELOCIDADE'] = df_fiscaltech.b.apply(lambda x: x[15:17]) #o certo seria [14:19], mas estamos padronizando
#sem casa decimal
df_fiscaltech['LATITUDE'] = df_fiscaltech.d.apply(lambda x: get_lat(x))
df_fiscaltech['LONGITUDE'] = df_fiscaltech.d.apply(lambda x: get_long(x)) #perceber que deixo um número antes do - de fora

df_fiscaltech.drop(['a','b','c','d','b0'], axis = 1, inplace = True)

# Salvando em csv
df_fiscaltech.to_csv('amostra_fiscaltech_tratada.csv', sep=';', index=False)

# Print
df_fiscaltech.sample(10)


,PLACA,TIPO,CODCET,CGT,DATAHORA,VELOCIDADE,LATITUDE,LONGITUDE
45,,3,0420071222,042,2019-06-01 03:48:00,60,-23.001678,-43.328644
36,,1,0420091111,042,2019-06-01 03:15:04,44,-23.003622,-43.32462
70,,1,0420181221,042,2019-06-01 03:15:01,11,-22.973304,-43.39867
77,,1,0420021123,042,2019-06-01 05:12:05,55,-23.000226,-43.353781
12,,1,0420071114,042,2019-06-01 01:12:01,57,-23.00141,-43.328091
34,,3,0420041213,042,2019-06-01 03:38:04,49,-23.000029,-43.339268
53,QNL6878,1,0420071123,042,2019-06-01 01:12:00,51,-23.001615,-43.328645
14,,3,0420081111,042,2019-06-01 03:18:00,09,-23.00303,-43.32414
18,,1,0420081112,042,2019-06-01 06:19:02,23,-23.00303,-43.32414
11,,1,0420081122,042,2019-06-01 03:32:04,43,-23.003226,-43.324684


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [46]:

#Tratamento da SITRAN
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran_raw = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)

df_sitran_split = separar_colunas_df(df_sitran_raw.copy())
df_sitran_split.head()



,a,b,c,d,e,f
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"2-22,90393-43,27340"
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"


In [47]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
        
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel


def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

df_sitran = pd.DataFrame()
df_sitran['CODCET'] = df_sitran_raw['x'].apply(lambda x: x[108:118])
df_sitran['DATAHORA'] = df_sitran_raw['x'].apply(lambda x: get_datahora(x))
df_sitran['PLACA'] = df_sitran_raw['x'].apply(lambda x: get_placa(x))
df_sitran['VELOCIDADE'] = df_sitran_raw['x'].apply(lambda x: get_velocidade(x))
df_sitran['CGT'] = df_sitran_raw['x'].apply(lambda x: x[108:111])
df_sitran['LATITUDE'] = df_sitran_raw['x'].apply(lambda x: x[216:225])
df_sitran['LONGITUDE'] = df_sitran_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_sitran.to_csv('amostra_sitran_tratada.csv', sep=';', index=False)
df_sitran.sample(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
59,0400141110,2019-03-01 10:52:29,,46,040,"-22,90393","-43,27340"
5,0400141110,2019-03-01 10:49:10,CXW7258,44,040,"-22,90393","-43,27340"
8,0400141110,2019-03-01 10:49:14,LLV1475,47,040,"-22,90393","-43,27340"
17,0400141110,2019-03-01 10:50:01,KRP7308,48,040,"-22,90393","-43,27340"
18,0400141110,2019-03-01 10:50:08,,32,040,"-22,90393","-43,27340"
53,0400141110,2019-03-01 10:52:16,KOQ9824,50,040,"-22,90393","-43,27340"
2,0400141110,2019-03-01 10:26:34,,49,040,"-22,90393","-43,27340"
48,0400141110,2019-03-01 10:51:56,,43,040,"-22,90393","-43,27340"
11,0400141110,2019-03-01 10:49:34,,53,040,"-22,90393","-43,27340"
40,0400141110,2019-03-01 10:51:23,,54,040,"-22,90393","-43,27340"


# SCP_INTVIA_0182018_201901_amostra.txt

In [22]:
#Tratamento de INTVIA
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia_raw = pd.read_csv('amostras/SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# separando as colunas
df_intvia_split = separar_colunas_df(df_intvia_raw)

df_intvia_split.head(20)



,a,b,c,d,e,f
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
5,-------1moto,0101201900004843,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
6,-------1moto,0101201900005239,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
7,-------1moto,0101201900005442,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
8,-------1moto,0101201900005947,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
9,-------1moto,0101201900010148,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"


In [71]:
def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date
def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

df_intvia = pd.DataFrame()
df_intvia['CODCET'] = df_intvia_raw['x'].apply(lambda x: x[108:118])
df_intvia['DATAHORA'] = df_intvia_raw['x'].apply(lambda x: get_datahora(x))
df_intvia['PLACA'] = df_intvia_raw['x'].apply(lambda x: get_placa(x))
df_intvia['VELOCIDADE'] = df_intvia_raw['x'].apply(lambda x: get_velocidade(x))
df_intvia['CGT'] = df_intvia_raw['x'].apply(lambda x: x[108:111])
df_intvia['LATITUDE'] = df_intvia_raw['x'].apply(lambda x: x[216:225])
df_intvia['LONGITUDE'] = df_intvia_raw['x'].apply(lambda x: x[225:234])


df_intvia.to_csv('amostras_tratadas/amostra_intvia_tratada.csv', sep=';', index=False)
df_intvia.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
81,0480091110,2019-01-01 00:20:37,,53,048,"-22,96635","-43,21941"
65,0480091110,2019-01-01 00:18:10,,48,048,"-22,96635","-43,21941"
30,0480091110,2019-01-01 00:06:33,,44,048,"-22,96635","-43,21941"
86,0480091110,2019-01-01 00:22:41,,23,048,"-22,96635","-43,21941"
92,0480091110,2019-01-01 00:24:35,LUP5085,45,048,"-22,96635","-43,21941"
